In [ ]:
# Check if the easydiffraction library is installed.
# If not, install it with the 'visualization' extras.
# Needed when running remotely (e.g. Colab) where the lib is absent.
import builtins
import importlib.util

if (hasattr(builtins, '__IPYTHON__') and
    importlib.util.find_spec('easydiffraction') is None):
    !pip install 'easydiffraction[visualization]'


# Pair Distribution Function: Ni, NPD

This example demonstrates a pair distribution function (PDF) analysis
of Ni, based on data collected from a constant wavelength neutron
powder diffraction experiment.

The dataset is taken from:
https://github.com/diffpy/cmi_exchange/tree/main/cmi_scripts/fitNiPDF

## Import Library

In [ ]:
import easydiffraction as ed

## Create Project

In [ ]:
project = ed.Project()

## Add Sample Model

In [ ]:
project.sample_models.add(name='ni')

In [ ]:
project.sample_models['ni'].space_group.name_h_m = 'F m -3 m'
project.sample_models['ni'].space_group.it_coordinate_system_code = '1'
project.sample_models['ni'].cell.length_a = 3.52387
project.sample_models['ni'].atom_sites.add(
    label='Ni',
    type_symbol='Ni',
    fract_x=0.0,
    fract_y=0.0,
    fract_z=0.0,
    wyckoff_letter='a',
    b_iso=0.5,
)

## Add Experiment

In [ ]:
data_path = ed.download_data(id=6, destination='data')

In [ ]:
project.experiments.add(
    name='pdf',
    data_path=data_path,
    sample_form='powder',
    beam_mode='constant wavelength',
    radiation_probe='neutron',
    scattering_type='total',
)

In [ ]:
project.experiments['pdf'].linked_phases.add(id='ni', scale=1.0)
project.experiments['pdf'].peak.damp_q = 0
project.experiments['pdf'].peak.broad_q = 0.03
project.experiments['pdf'].peak.cutoff_q = 27.0
project.experiments['pdf'].peak.sharp_delta_1 = 0.0
project.experiments['pdf'].peak.sharp_delta_2 = 2.0
project.experiments['pdf'].peak.damp_particle_diameter = 0

## Select Fitting Parameters

In [ ]:
project.sample_models['ni'].cell.length_a.free = True
project.sample_models['ni'].atom_sites['Ni'].b_iso.free = True

In [ ]:
project.experiments['pdf'].linked_phases['ni'].scale.free = True
project.experiments['pdf'].peak.broad_q.free = True
project.experiments['pdf'].peak.sharp_delta_2.free = True

## Run Fitting

In [ ]:
project.analysis.current_calculator = 'pdffit'
project.analysis.fit()

## Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='pdf', show_residual=True)